In [1]:
import itertools
import pandas as pd
import cobra.flux_analysis
from cobra import Metabolite
import time
import numpy as np
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA, knockout_FBA_w_tasks

from functools import partial
from src.FBA_scripts.met_task_functions import constrain_model, read_tasks


"""A mess of a document with different code cells.
Good to to use for any testing that involves the Human1 model as it takes some time to load in."""

def load_model(model_path: str) -> list:
    model = cobra.io.read_sbml_model(model_path)
    model_list = constrain_model(model, ALLMETSIN=True)
    #Removes unused model.
    model_list[2] = None
    return model_list


def get_unique_gene_combs(combs: pd.DataFrame) -> pd.DataFrame:

    df = pd.DataFrame(combs['gene_ids'], columns=['gene_ids'])
    f = list(np.unique(df))
    d = {}
    n=0
    for l in f:
        d[n] = [l]
        n += 1

    return pd.DataFrame.from_dict(d, orient='index', columns=['gene_ids'])


def produce_ind_combs(model_genes: list) -> pd.DataFrame:
    ind_data = pd.read_table('C:/Users/Sigve/Genome_Data/SNP_data/1000_genomes/result_chrom_22.tsv', index_col=0)
    missense_snps = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNPs_missense.tsv', index_col=0)

    missense_snps['gene_id'] = missense_snps['gene_id'].apply(lambda x: x.split('.')[0])
    merged_df = pd.merge(left=missense_snps, right=ind_data, how='left', left_on='chrom_pos', right_on='pos')

    res_dict = {}

    for column in merged_df:
        res_dict[column] = [merged_df['variant_name'][merged_df[column] == 2].to_list(), set(merged_df['gene_id'][merged_df[column] == 2].to_list())]

    results = pd.DataFrame.from_dict(res_dict, orient='index', columns=['snp_list', 'gene_ids'])
    results['gene_ids'] = results['gene_ids'].apply(lambda x: list(x.intersection(model_genes)))
    results = results[results['gene_ids'].map(lambda x: len(x)) > 0]

    return results

In [ ]:
start_time = time.time()
# ['blood']
tissue_list = ['brain', 'heart', 'kidney', 'liver', 'muscle', 'nerve', 'pancreas', 'thyroid']
model_base_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/{0}.xml'

for tissue in tissue_list: 

    model_list = load_model(model_base_path.format(tissue))
    model_genes = [gene.id for gene in model_list[0].genes]

    task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv', model_list)
    ind_combs = produce_ind_combs(model_genes)
    ind_combs['gene_ids'].apply(lambda x: x.sort())

    unique_combs = get_unique_gene_combs(ind_combs)

    results = parallelize_dataframe(unique_combs, partial(combinations_subset, partial(knockout_FBA_w_tasks, task_list, model_list)), 8)

    results['gene_ids'] = results['gene_ids'].apply(lambda x: ';'.join(x))
    results['solution'] = results['results'].apply(lambda x: x[0])
    results['tasks_results'] = results['results'].apply(lambda x: x[1:])

    ind_combs['gene_ids'] = ind_combs['gene_ids'].apply(lambda x: ';'.join(x))
    ind_combs.reset_index(drop=False, inplace=True)
    ind_combs['ind_id'] = ind_combs['index']

    results = pd.merge(left=ind_combs, right=results, how='right', left_on='gene_ids', right_on='gene_ids')
    results['gene_ids'] = results['gene_ids'].apply(lambda x: x.split(';'))




    results[['ind_id', 'solution', 'tasks_results']].to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/ind_results/ind_all_start_stop_chrom_{0}.tsv'.format(tissue), sep='\t')


end_time = time.time()

print('Total time: %.6f seconds' % (end_time - start_time))

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpr_d2uv5b.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpomnktrt8.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmp7a5m4470.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpvjj4pq9v.lp
Reading time = 0.07 seconds
: 5321 rows, 14488 columns, 63246 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpnkkgr6vc.lp
Reading time = 0.06 seconds
: 5321 rows, 14488 columns, 63246 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmp3jsrusu4.lp
Reading time = 0.06 seconds
: 5321 rows, 14488 columns, 6

In [2]:
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/brain.xml'
model = cobra.io.read_sbml_model(model_file_path)

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic


In [84]:
model_genes = [gene.id for gene in model.genes]
combs = produce_ind_combs(model_genes)

In [ ]:
print(combs.shape[0])


In [2]:
# For new one

start_time = time.time()

tissue_list =  ['brain']#, 'skin', 'spleen', 'adipose_tissue', 'adrenal_gland', 'uterus']#['pancreas', 'thyroid', 'blood', 'brain', 'heart', 'kidney', 'liver', 'muscle', 'nerve']

ind_data = pd.read_table('C:/Users/Sigve/Genome_Data/results/ind_combinations/all_missense_comb_ho.tsv', index_col=0)
ind_data['gene_ids'] = ind_data['gene_ids'].apply(lambda x: x[2:-2].split(';'))

# Just cosmetic editing, not necessary for functions
ind_data['sample_ids'] = ind_data['sample_ids'].apply(lambda x: ';'.join(x[2:-2].split("', '")))

for tissue in tissue_list:

    non_essential_genes = pd.read_table('C:/Users/Sigve/Genome_Data/results/model_tests/{0}_non_ess_genes_ext.tsv'.format(tissue), index_col=0)['gene_ids'].tolist()

    model_list = load_model('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/{0}.xml'.format(tissue))
    model_genes = [gene.id for gene in model_list[0].genes] # Use non essential genes instead

    results = ind_data.copy()
    results['gene_ids'] = results['gene_ids'].apply(lambda x: list(set(x).intersection(non_essential_genes)))
    results = results[results['gene_ids'].map(lambda x: len(x)) > 0]

    task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv', model_list)

    results = parallelize_dataframe(results, partial(combinations_subset, partial(knockout_FBA_w_tasks, task_list, model_list)), 12)

    results.loc[-1] = ['REF', [], knockout_FBA_w_tasks(task_list, model_list, [])]
    results['gene_ids'] = results['gene_ids'].apply(';'.join)
    results['solution'] = results['results'].apply(lambda x: round(x[0], 3))
    results['tasks_results'] = results['results'].apply(lambda x: x[1:] if not all(x[1:]) else ['ALL PASS'])

    results.reset_index(inplace=True, drop=True)
    results[['sample_ids', 'gene_ids', 'solution', 'tasks_results']].to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/ind_results/extensive/ind_all_chrom_sel_{0}_ho_e.tsv'.format(tissue), sep='\t')


end_time = time.time()

print('Total time: %.6f seconds' % (end_time - start_time))


Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpu0isfjqv.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmppnh2euux.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpzmdozqfj.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Total time: 7340.737645 seconds


In [237]:
# Constrain testing below:

model_list = load_model('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/lung.xml')
task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv', model_list)
result_list = []

Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpycqh6gsq.lp
Reading time = 0.07 seconds
: 5665 rows, 15720 columns, 68914 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpnt71_an7.lp
Reading time = 0.07 seconds
: 5665 rows, 15720 columns, 68914 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpu0_s0qj_.lp
Reading time = 0.07 seconds
: 5665 rows, 15720 columns, 68914 nonzeros


In [240]:
result_list.append(knockout_FBA_w_tasks(task_list, model_list, ['ENSG00000197249']))

In [241]:
print(result_list[-1])

[82.17122442635676, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [242]:
task = task_list[-1]
for subset in [task[0], task[1]]:
    for rx in subset:
        if rx == 'ALLMETSIN':
            # Adds boundary metabolites for other reactions when ALLMETSIN is used
            for r in subset[1:]:
                for m2 in r.metabolites:
                    for r2 in m2.reactions:
                        if r2.boundary and r2.id != r.id:
                            # Could also just remove the reactions, or set them 0, 0
                            r2.add_metabolites({Metabolite(
                                                m2.id[:-4] + 'x[x]',
                                                formula=m2.formula,
                                                name=' '.join(m2.name.split(' ')[:-1]) + ' [Boundary]',
                                                compartment='x'): 1})
            continue
        model_list[3].add_reaction(rx)

In [243]:
print(knockout_FBA(model_list[3], []))

<Solution 11.047 at 0x1b98b9092e0>


In [244]:
# Add input constraints on glucose and threonine and reduce the minimum biomass production
model_list[3].reactions.get_by_id('ess_57_out_1').lower_bound = 0
print(model_list[3].reactions.get_by_id('ess_57_out_1').bounds)

model_list[3].reactions.get_by_id('ess_57_in_33').upper_bound = 1
print(model_list[3].reactions.get_by_id('ess_57_in_33').bounds)
print(knockout_FBA(model_list[3], []))

model_list[3].reactions.get_by_id('ess_57_in_10').upper_bound = 0.1
print(model_list[3].reactions.get_by_id('ess_57_in_10').bounds)
print(knockout_FBA(model_list[3], []))

(0, 1000.0)
(0.0, 1)
<Solution 11.047 at 0x1b98b90ddf0>
(0.0, 0.1)
<Solution 0.329 at 0x1b98b90d520>


In [245]:
# Add L-Lactate export constraint
model_list[3].reactions.get_by_id('HMR_9135').upper_bound = 0.1
print(model_list[3].reactions.get_by_id('HMR_9135').bounds)
print(knockout_FBA(model_list[3], []))

(0, 0.1)
<Solution 0.329 at 0x1b98b908310>


In [198]:
for i in range(1,31):
    model_list[3].reactions.get_by_id('ess_57_in_{0}'.format(i)).upper_bound = 1000

In [251]:
res = knockout_FBA(model_list[3], [])
res2 = knockout_FBA(model_list[3], ['ENSG00000197249'])
print(res)

<Solution 0.329 at 0x1b98b90c040>


In [249]:
mets = ['m01716s[s]', 'm01716c[c]', 'm01716m[m]', 'm02403s[s]', 'm02403c[c]', 'm02403m[m]', 'm02403p[p]']
model = model_list[3]
rx = []
for m in mets:
    for r in model.metabolites.get_by_id(m).reactions:
        rx.append(r)

rx = set(rx)

for r in rx:
    if res.fluxes.at[r.id] != 0:
        print(r.id + ': ' + r.reaction + ' ' + str(res.fluxes.at[r.id]))

HMR_4280: m02039m[m] + m02553m[m] + m02819m[m] <=> m02403m[m] + m02552m[m] 409.509778544807
HMR_9135: m02403s[s] -->  0.1
HMR_3007: m02039c[c] + m02403c[c] <=> m02039p[p] + m02403p[p] -0.0004798369432042753
HMR_6048: m01410s[s] + m02403c[c] <=> m01410c[c] + m02403s[s] 1000.0
HMR_6049: m02403c[c] + m02833s[s] <=> m02403s[s] + m02833c[c] -999.9
HMR_4281: m02039p[p] + m02553p[p] + m02819p[p] <=> m02403p[p] + m02552p[p] 0.0004798369432042753
HMR_4388: m02039c[c] + m02553c[c] + m02819c[c] <=> m02403c[c] + m02552c[c] -409.41025838175017
HMR_5411: m02039c[c] + m02403c[c] <=> m02039m[m] + m02403m[m] -409.509778544807


In [223]:
"""
HMR_6049: m02403c[c] + m02833s[s] <=> m02403s[s] + m02833c[c] 0.006393322022483699
HMR_9135: m02403s[s] -->  0.006393322022483699
HMR_4280: m02039m[m] + m02553m[m] + m02819m[m] <=> m02403m[m] + m02552m[m] -0.009737132747913484
HMR_4281: m02039p[p] + m02553p[p] + m02819p[p] <=> m02403p[p] + m02552p[p] 0.016130454770397182
L_LACtcm: m02403c[c] --> m02403m[m] 0.009737132747913484
HMR_3007: m02039c[c] + m02403c[c] <=> m02039p[p] + m02403p[p] -0.016130454770397182
----------------------------------------

"""

'\nHMR_6049: m02403c[c] + m02833s[s] <=> m02403s[s] + m02833c[c] 0.006393322022483699\nHMR_9135: m02403s[s] -->  0.006393322022483699\nHMR_4280: m02039m[m] + m02553m[m] + m02819m[m] <=> m02403m[m] + m02552m[m] -0.009737132747913484\nHMR_4281: m02039p[p] + m02553p[p] + m02819p[p] <=> m02403p[p] + m02552p[p] 0.016130454770397182\nL_LACtcm: m02403c[c] --> m02403m[m] 0.009737132747913484\nHMR_3007: m02039c[c] + m02403c[c] <=> m02039p[p] + m02403p[p] -0.016130454770397182\n----------------------------------------\n\n'

In [260]:
merged = pd.DataFrame(res.fluxes).merge(res2.fluxes, left_index=True, right_index=True)
merged['comparison'] = 0 if merged['fluxes_y'] is not merged['fluxes_x'] else 1
print(merged.head())

print(merged[merged['comparison'] == 1])


          fluxes_x  fluxes_y  comparison
HMR_3905       0.0       0.0           0
HMR_3907       0.0       0.0           0
HMR_4097       0.0       0.0           0
HMR_4099       0.0       0.0           0
HMR_4108       0.0       0.0           0
Empty DataFrame
Columns: [fluxes_x, fluxes_y, comparison]
Index: []


In [3]:
# Results Reading
tissue_list =  ['brain']#['skin', 'spleen', 'adipose_tissue', 'adrenal_gland', 'uterus', 'pancreas', 'thyroid', 'blood', 'brain', 'heart', 'kidney', 'liver', 'muscle', 'nerve']

for tissue in tissue_list:
    res = pd.read_table('C:/Users/Sigve/Genome_Data/results/ind_results/extensive/ind_all_chrom_sel_{0}_ho_e.tsv'.format(tissue), index_col=0)
    res['tasks_results'] = res['tasks_results'].apply(lambda x: [int(i) for i in x[1:-1].split(', ')] if x[2:-2] != 'ALL PASS' else ['ALL PASS'])
    res['solution'] = res['solution'].apply(lambda x: round(x, 3))

    ref_sol = res['solution'].iloc[-1]#Change to 'REF'
    res['chk'] = res['tasks_results'].apply(lambda x: not all(x))

    non_nom = res[((res['solution'] != ref_sol) | res['chk']) | (res['sample_ids'] == 'REF')]
    non_nom[['sample_ids', 'solution', 'tasks_results']].to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/ind_results/filtered/ind_all_chrom_sel_{0}_ho_e_f.tsv'.format(tissue), sep='\t')
    print(tissue + ': ' + str(res.shape[0]))
    print(tissue + ': ' + str(non_nom.shape[0]) + ' results with damage.')
    print('-----------------------------------')

brain: 2549
brain: 2547 results with damage.
-----------------------------------


In [304]:
model = cobra.io.read_sbml_model('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/pancreas.xml')

In [314]:
print(knockout_FBA(model, ['ENSG00000110536']))

<Solution 0.000 at 0x1b90dd0ee20>


In [9]:
# Get all (non)essential genes. SLOW, just use MATLAB for this
start_time = time.time()
tissue_list =  ['skin', 'spleen', 'adipose_tissue', 'adrenal_gland', 'uterus', 'pancreas', 'thyroid', 'blood', 'heart', 'kidney', 'liver', 'muscle', 'nerve'] #['brain']
for tissue in tissue_list:

    model_list = load_model('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/{0}.xml'.format(tissue))
    task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv', model_list)

    essential_genes = [gene.id for gene in cobra.flux_analysis.find_essential_genes(model_list[0], processes=12)]
    extensive_genes = [[[gene]] for gene in [g.id for g in model_list[0].genes] if gene not in essential_genes]

    extensive_genes_df = pd.DataFrame(extensive_genes, columns=['gene_ids'])

    results = parallelize_dataframe(extensive_genes_df, partial(combinations_subset, partial(knockout_FBA_w_tasks, task_list, model_list)), 12)

    essential_df = pd.DataFrame([gene[0] for gene in results[results['results'].map(lambda x: not all(x))]['gene_ids'].tolist()] + essential_genes, columns=['gene_ids'])
    non_essential_df = results[results['results'].map(lambda x: all(x))]['gene_ids'].apply(lambda x: x[0]).reset_index(drop=True)

    essential_df.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/model_tests/essential_genes/{0}_essential.tsv'.format(tissue), sep='\t')
    non_essential_df.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/model_tests/essential_genes/{0}_non_essential.tsv'.format(tissue), sep='\t')

    print('Genes in the model: ' + str(len(model_list[0].genes)))
    print('Essential genes: ' + str(essential_df.shape[0]))
    print('Non-essential genes: ' + str(non_essential_df.shape[0]))
    print('Total number out: ' + str(essential_df.shape[0] + non_essential_df.shape[0]))

end_time = time.time()
print('Total time: %.6f seconds' % (end_time - start_time))

Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpoxqb0b4_.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpkjw6h3q3.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmp64q0gnpg.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Genes in the model: 2451
Essential genes: 288
Non-ssential genes: 2
Total number out: 290


In [12]:
# combination generator for individual samples.

tissue = 'liver'
sample='HG03385'
input_string = "{'ENSG00000104524;ENSG00000125508;ENSG00000159399;ENSG00000125255;ENSG00000159131;ENSG00000184307;ENSG00000167889;ENSG00000131446;ENSG00000162571;ENSG00000137491;ENSG00000198704;ENSG00000155093;ENSG00000170242;ENSG00000103657;ENSG00000277494;ENSG00000101349;ENSG00000070748;ENSG00000118369;ENSG00000129951;ENSG00000180917;ENSG00000124767;ENSG00000160190;ENSG00000003987;ENSG00000198833;ENSG00000136960;ENSG00000127415;ENSG00000139428;ENSG00000115020;ENSG00000177169;ENSG00000197355;ENSG00000185420;ENSG00000107611;ENSG00000183044;ENSG00000154229;ENSG00000135744;ENSG00000123989;ENSG00000130653;ENSG00000165841;ENSG00000144741;ENSG00000095303;ENSG00000159339;ENSG00000203857;ENSG00000117461;ENSG00000103569;ENSG00000172572;ENSG00000007541;ENSG00000155561;ENSG00000175054;ENSG00000203837;ENSG00000169071;ENSG00000244122;ENSG00000214013;ENSG00000128655;ENSG00000178814;ENSG00000184983;ENSG00000120265;ENSG00000123453;ENSG00000100156;ENSG00000112053;ENSG00000182670;ENSG00000117143;ENSG00000102595;ENSG00000130164;ENSG00000114771;ENSG00000166819;ENSG00000171105;ENSG00000174640;ENSG00000197586;ENSG00000124253;ENSG00000108592;ENSG00000160285;ENSG00000112096;ENSG00000116771;ENSG00000257335;ENSG00000146904;ENSG00000198842;ENSG00000175548;ENSG00000127329;ENSG00000115687;ENSG00000146414;ENSG00000168411;ENSG00000105205;ENSG00000137198;ENSG00000168350;ENSG00000184860;ENSG00000084110;ENSG00000163162;ENSG00000142185;ENSG00000163482;ENSG00000071967;ENSG00000151692;ENSG00000145321;ENSG00000064270;ENSG00000085998;ENSG00000142973;ENSG00000115464;ENSG00000138376;ENSG00000125246;ENSG00000165140;ENSG00000134184;ENSG00000136877;ENSG00000112499;ENSG00000112303;ENSG00000134013;ENSG00000106278;ENSG00000147684;ENSG00000126432;ENSG00000109861;ENSG00000090402;ENSG00000155657;ENSG00000173083;ENSG00000023839;ENSG00000125779;ENSG00000198246;ENSG00000157881;ENSG00000160446;ENSG00000124370;ENSG00000108839;ENSG00000144401;ENSG00000115884;ENSG00000068001;ENSG00000084674;ENSG00000161513;ENSG00000146373;ENSG00000156510;ENSG00000124006;ENSG00000162390;ENSG00000154305;ENSG00000130035;ENSG00000275342;ENSG00000158104;ENSG00000040933;ENSG00000179593;ENSG00000136381;ENSG00000181085;ENSG00000138074;ENSG00000060140;ENSG00000274391;ENSG00000165434;ENSG00000065534;ENSG00000170485;ENSG00000162402;ENSG00000104808;ENSG00000120156;ENSG00000090861;ENSG00000106346;ENSG00000115705;ENSG00000197594;ENSG00000139988;ENSG00000114923;ENSG00000170835;ENSG00000002726;ENSG00000206077;ENSG00000188833;ENSG00000156413;ENSG00000116711;ENSG00000070915;ENSG00000105675;ENSG00000123552;ENSG00000116791;ENSG00000069020;ENSG00000187486;ENSG00000117215;ENSG00000182224;ENSG00000187630;ENSG00000162882;ENSG00000099821;ENSG00000164543;ENSG00000162365;ENSG00000105929;ENSG00000169021;ENSG00000186204;ENSG00000106648;ENSG00000036565;ENSG00000182793;ENSG00000105509;ENSG00000148356;ENSG00000100116;ENSG00000129204;ENSG00000213341;ENSG00000171766;ENSG00000012963;ENSG00000156463;ENSG00000147576;ENSG00000105379;ENSG00000096006;ENSG00000165996;ENSG00000198691;ENSG00000102699;ENSG00000123612;ENSG00000094963;ENSG00000107779;ENSG00000188467;ENSG00000083444;ENSG00000076555;ENSG00000006071;ENSG00000072195;ENSG00000154227;ENSG00000136628;ENSG00000176463;ENSG00000169519;ENSG00000146070;ENSG00000104055;ENSG00000138061;ENSG00000183747;ENSG00000011083;ENSG00000166349;ENSG00000138744;ENSG00000173540;ENSG00000174607;ENSG00000250305;ENSG00000182601;ENSG00000198075;ENSG00000143921;ENSG00000113924;ENSG00000140534;ENSG00000128242;ENSG00000180251;ENSG00000177666;ENSG00000175003;ENSG00000183921;ENSG00000141012;ENSG00000115977;ENSG00000099377;ENSG00000129353;ENSG00000084754;ENSG00000163803;ENSG00000071909;ENSG00000115275;ENSG00000099817;ENSG00000105355;ENSG00000079393;ENSG00000157326;ENSG00000008128;ENSG00000132256;ENSG00000277893;ENSG00000177669;ENSG00000138030;ENSG00000259075;ENSG00000115159;ENSG00000114480;ENSG00000081479;ENSG00000053371;ENSG00000164116;ENSG00000172296;ENSG00000168038;ENSG00000143776;ENSG00000072756;ENSG00000186716;ENSG00000139304;ENSG00000184470;ENSG00000131864;ENSG00000167676;ENSG00000178921;ENSG00000090661;ENSG00000143499;ENSG00000058866;ENSG00000139144;ENSG00000136169;ENSG00000038002;ENSG00000141503;ENSG00000183421;ENSG00000218823;ENSG00000170222;ENSG00000171094;ENSG00000132677;ENSG00000085511;ENSG00000179913;ENSG00000155850;ENSG00000196616;ENSG00000124568;ENSG00000086015;ENSG00000164363;ENSG00000119121;ENSG00000198074;ENSG00000177239;ENSG00000243477;ENSG00000124588;ENSG00000162551;ENSG00000117834;ENSG00000135318;ENSG00000163751;ENSG00000172543;ENSG00000140598;ENSG00000105647;ENSG00000069943;ENSG00000135587;ENSG00000137574;ENSG00000187980;ENSG00000146426;ENSG00000108515;ENSG00000156194;ENSG00000111058;ENSG00000070669;ENSG00000105426;ENSG00000100344;ENSG00000171124;ENSG00000170266;ENSG00000135220;ENSG00000112739;ENSG00000147804;ENSG00000044524;ENSG00000139344;ENSG00000187714;ENSG00000198077;ENSG00000134070;ENSG00000171298;ENSG00000106123;ENSG00000116539;ENSG00000137261;ENSG00000137124;ENSG00000124181;ENSG00000149124;ENSG00000134398;ENSG00000104635;ENSG00000188906;ENSG00000128829;ENSG00000288702;ENSG00000162408;ENSG00000198099;ENSG00000066230;ENSG00000135372;ENSG00000072609;ENSG00000164294;ENSG00000125166;ENSG00000171234;ENSG00000135090;ENSG00000065613;ENSG00000179299;ENSG00000143772;ENSG00000080031;ENSG00000143845;ENSG00000158828;ENSG00000170231;ENSG00000100577;ENSG00000016391;ENSG00000140284;ENSG00000125780;ENSG00000073711;ENSG00000159082;ENSG00000132746;ENSG00000174227;ENSG00000177200;ENSG00000116353;ENSG00000134242;ENSG00000132437;ENSG00000154415;ENSG00000157765;ENSG00000122025;ENSG00000100714;ENSG00000128683;ENSG00000103485;ENSG00000178685;ENSG00000186334;ENSG00000109854;ENSG00000095015;ENSG00000127947;ENSG00000138823;ENSG00000248099;ENSG00000088035;ENSG00000126883;ENSG00000093072;ENSG00000197323;ENSG00000133256;ENSG00000063176;ENSG00000131400;ENSG00000130589;ENSG00000142731;ENSG00000169710;ENSG00000051341;ENSG00000272333;ENSG00000198355;ENSG00000095321;ENSG00000109323;ENSG00000149089;ENSG00000198569;ENSG00000115850;ENSG00000049860;ENSG00000198610;ENSG00000113492;ENSG00000135913;ENSG00000130957;ENSG00000083290;ENSG00000067365;ENSG00000164867;ENSG00000172890;ENSG00000257594;ENSG00000176920;ENSG00000164303;ENSG00000077463;ENSG00000182272;ENSG00000168827;ENSG00000008300;ENSG00000166035;ENSG00000177191;ENSG00000060237;ENSG00000111684;ENSG00000070778;ENSG00000003137;ENSG00000172071;ENSG00000112294;ENSG00000140057;ENSG00000151148;ENSG00000171903;ENSG00000156222;ENSG00000134216;ENSG00000170439;ENSG00000132517;ENSG00000074803;ENSG00000132182;ENSG00000153207'}"

gene_list = input_string[2:-2].split(';')
#gene_list = [gene.split(': ')[0] for gene in ''[2:-2].split(input_string)]

print('Number of genes in combination: ' + str(len(gene_list)))

non_essential_genes = pd.read_table('C:/Users/Sigve/Genome_Data/results/model_tests/essential_genes/{0}_non_essential.tsv'.format(tissue), index_col=0)['gene_ids'].tolist()
gene_list = list(set(gene_list).intersection(non_essential_genes))
print('Number of genes in combination after removal of essential and non model genes: ' + str(len(gene_list)))

combs = []
for L in range(2, 2 + 1):
    for subset in itertools.combinations(gene_list, L):
        combs.append(';'.join(subset))

print('Number of combinations: ' + str(len(combs)))
combinations_df = pd.DataFrame(combs, columns=['gene_ids'])
combinations_df.reset_index(inplace=True)
combinations_df.rename(columns={'index': 'sample_ids'}, inplace=True)
combinations_df['sample_ids']=combinations_df['sample_ids'].apply(str)
combinations_df['sample_ids']=combinations_df['sample_ids'].apply(lambda x: set([x]))
combinations_df['gene_ids']=combinations_df['gene_ids'].apply(lambda x: set([x]))
combinations_df.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/ind_combinations/{0}_{1}_combinations.tsv'.format(sample, tissue), sep='\t')

Number of genes in combination: 411
Number of genes in combination after removal of essential and non model genes: 215
Number of combinations: 23005
